In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 60
pd.options.display.max_rows = 500
import warnings
warnings.filterwarnings('ignore')

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import os

import numpy as np

In [3]:
frac = 0.02

In [4]:
# Adulto equivalente. Cuanto cuesta la manutencion de las personas segun sexo y edad.
ad_eq = pd.read_csv('./../data/info/adulto_eq.csv')

#Importar canasta basica regional deflac
CB_ipc = pd.read_csv('./../data/info/CB_Reg_defl.csv')

# Load radio ref. Merge regiones.
# Anything that is AGLOMERADO 33 should be region Gran Buenos Aires

radio_ref = pd.read_csv('./../data/info/radio_ref.csv').merge(pd.read_csv('./../data/info/prov_regs.csv'), how = 'left')

radio_ref = pd.read_csv('./../data/info/radio_ref.csv')#.merge(aglo_labels)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
radio_ref = radio_ref.merge(dpto_region)

# DPTO_Region = radio_ref[['DPTO', 'Region']].drop_duplicates()

In [5]:
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
DPTO_Region = radio_ref[['DPTO', 'Region']].drop_duplicates()

## 1.  Resultados estaticos (se toma 1 año)

In [6]:
import glob
# path ='./data/RFReg_' # use your path
# path ='./../../encuestador-de-hogares/data/yr_samples/RFReg_' # use your path
path ='./../data/yr_samples/RFReg_'

allFiles = []
for year in [str(s) for s in range(2019, 2021)]:
    allFiles += glob.glob(path +str(frac)+ '*'+str(year)+'*_ARG.csv')
    # Estos son los archivos que se usan para tener una figura estatica, corte donde no importa evol. temporal.

sorted(allFiles)[-4:] # ultimo anio


['./../data/yr_samples/RFReg_0.02_2019-11-15_ARG.csv',
 './../data/yr_samples/RFReg_0.02_2020-02-15_ARG.csv',
 './../data/yr_samples/RFReg_0.02_2020-05-15_ARG.csv',
 './../data/yr_samples/RFReg_0.02_2020-08-15_ARG.csv']

In [7]:
# years = np.unique([int(f[-14:-10]) for f in  allFiles])
years = np.unique([int(f[-18:-14]) for f in  allFiles[-4:]]) # ultimo anio 
# f[-18:-14] # Depende de la convencion para nombrar el archivo

if len(years) == 1:
    yr_label = str(years[0])
else:
    yr_label = '-'.join([str(years.min()), str(years.max())])
    
yr_label

'2019-2020'

In [8]:

df_parts = []
for quarter_Xy_file in sorted(allFiles)[-4:]:# ultimo anio
    df_Q = pd.read_csv(quarter_Xy_file, 
                           usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID','RADIO_REF_ID', 'CONDACT', 'CAT_INAC', 'CAT_OCUP',
                                      'IX_TOT', 'H16', 'H15','P47T', 'P03','P02', 'P09','P10', 'DPTO'])
    df_Q['ANO4'] = int(quarter_Xy_file[-18:-14])
    q = quarter_Xy_file[-18:-8]; print(q)
    df_Q['Q'] = q
    df_parts += [df_Q]

df = pd.concat(df_parts)
del df_Q


2019-11-15
2020-02-15
2020-05-15
2020-08-15


In [9]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# # Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:7]

[('df', 722765073),
 ('radio_ref', 17219219),
 ('dpto_region', 120394),
 ('CB_ipc', 69912),
 ('DPTO_Region', 44574),
 ('ad_eq', 5472),
 ('ProgressBar', 1064)]

In [10]:

columnas_pesos = ['P47T']
df[columnas_pesos] = np.power(10, df[columnas_pesos]) - 1

# # df.PERSONA_REF_ID.nunique()
# 1756665 - 891932 - 884035 = -19302

# use means:
# ddf = dd.from_pandas(df, npartitions=50);
# with ProgressBar():
#     df = ddf.groupby(['PERSONA_REF_ID','Q']).mean().compute() 
## Hay una pequena cantidad de duplicados, gente que fue sampleada mas de una vez, y a las que el modelo les dio el mismo resultado en repetidas veces.
# El unico problemita que dan es que los hogares donde estan se agruparian en uno solo. 
# Quizas se corrige sacando el sampleo con replacement.
# Por ahora ignoro el detalle

# Editar columnas
df['P10'] = 2 - df['P10']
df['P09'] = df.P09.replace(5, 4) #Polimodal tomado como secundario 

# df = df.astype(int)
df['P0910'] = df.P09.astype(str) + df.P10.astype(str)
df['Grupo_Etario_3'] = pd.cut(df.P03, np.arange(-1, 80, 3))#.round(-1)
df['Grupo_Etario_INDEC'] = pd.cut(df.P03, np.array([0, 14, 29, 64, 110]))#.round(-1)
df['Grupo_Etario_q10'] = pd.cut(df.P03, np.array([-0.001, 5.0,  11.0, 17.0, 23.0, 29.0, 36.0, 44.0, 53.0, 65.0, 110.0]))#.round(-1)

df = df.reset_index()


In [11]:
# # Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:7]


[('df', 943513579),
 ('radio_ref', 17219219),
 ('dpto_region', 120394),
 ('CB_ipc', 69912),
 ('DPTO_Region', 44574),
 ('ad_eq', 5472),
 ('ProgressBar', 1064)]

In [12]:
df.groupby('Q').size()

Q
2019-11-15    884200
2020-02-15    892073
2020-05-15    892073
2020-08-15    892073
dtype: int64

In [13]:
df.shape

(3560419, 22)

In [16]:
df_cb = df.merge(ad_eq).merge(DPTO_Region).merge(CB_ipc)

df_cb_hogares = df_cb.groupby(['HOGAR_REF_ID', 'Q'])[['P47T','CBA', 'CBT', 'CB_EQUIV']].sum()
del df_cb # Ahorrar memoria

df_cb_hogares['Pobreza'] = df_cb_hogares['P47T'] < df_cb_hogares['CBT']
df_cb_hogares['Indigencia'] = df_cb_hogares['P47T'] < df_cb_hogares['CBA']
pobreza_hogares = df_cb_hogares[['P47T','CBA','CBT', 'CB_EQUIV','Pobreza', 'Indigencia']].reset_index()
pobreza_hogares['gap_pobreza'] = pobreza_hogares.P47T - pobreza_hogares.CBT
pobreza_hogares['gap_indigencia'] = pobreza_hogares.P47T - pobreza_hogares.CBA
pobreza_hogares = pobreza_hogares.rename(columns = {'P47T': 'P47T_hogar'})

# df = df.sample(25000)
data = df.merge(pobreza_hogares, on = ['HOGAR_REF_ID', 'Q'])#, how = 'left')
del df; del pobreza_hogares # Ahorrar memoria

data = data.rename(columns = {'P47T': 'P47T_persona'})

# data = data
data = data.merge(radio_ref[['RADIO_REF_ID', 'IDFRAC', 'PROV', 'NOMPROV', 'AGLOMERADO']].drop_duplicates())

if not os.path.exists('./../data/Pobreza/'):
    os.makedirs('./../data/Pobreza/')
filename = './../data/Pobreza/pobreza_'+'_'.join([str(frac), yr_label])+'.csv'
data.to_csv(filename, index = False)


In [17]:
data.shape

(3560419, 34)

In [18]:
# # Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:7]


[('data', 1512924283),
 ('df_cb_hogares', 46267873),
 ('radio_ref', 17444317),
 ('dpto_region', 120394),
 ('CB_ipc', 69912),
 ('DPTO_Region', 44574),
 ('ad_eq', 5472)]

## 2. Resultados series de tiempo (se computa para todos y cada trimestre)

In [19]:
import glob
# path ='./data/RFReg_' # use your path
# path ='./../data/yr_samples/RFReg_' # use your path
# path ='./../../encuestador-de-hogares/data/yr_samples/RFReg_' # use your path
path ='./../../indice-pobreza-ExactasUBA/data/yr_samples/RFReg_' # use your path
path ='./../data/yr_samples/RFReg_'


allFiles = []
for year in [str(s) for s in range(2019, 2021)]: ## Importante poner el anio de comienzo de la serie
    # Estos archivos se computan en la notebook tipo 04c - Predict using trained models- Empleo Trimestral.ipynb
    allFiles += glob.glob(path +str(frac)+'_'+ year +'*.csv')
sorted(allFiles)

['./../data/yr_samples/RFReg_0.02_2019-02-15_ARG.csv',
 './../data/yr_samples/RFReg_0.02_2019-05-15_ARG.csv',
 './../data/yr_samples/RFReg_0.02_2019-08-15_ARG.csv',
 './../data/yr_samples/RFReg_0.02_2019-11-15_ARG.csv',
 './../data/yr_samples/RFReg_0.02_2020-02-15_ARG.csv',
 './../data/yr_samples/RFReg_0.02_2020-05-15_ARG.csv',
 './../data/yr_samples/RFReg_0.02_2020-08-15_ARG.csv']

In [20]:
for quarter_Xy_file in sorted(allFiles):
    if not os.path.exists('./../data/Pobreza/'):
        os.makedirs('./../data/Pobreza/')
        
    q = quarter_Xy_file[-18:-8]
    filename = './../data/Pobreza/pobreza_'+'_'.join([str(frac), 'q'+q])+'.csv'
    print(q)
    if not os.path.exists('./../data/Pobreza/pobreza_'+'_'.join([str(frac), 'q'+q])+'.csv'):

        df = pd.read_csv(quarter_Xy_file, 
                               usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID','RADIO_REF_ID', 'CONDACT', 'CAT_INAC', 'CAT_OCUP',
                                          'IX_TOT', 'H16', 'H15','P47T', 'P03','P02', 'P09','P10', 'DPTO'])
        df['ANO4'] = int(quarter_Xy_file[-18:-14])
        df['Q'] = q

        columnas_pesos = ['P47T']
        df[columnas_pesos] = np.power(10, df[columnas_pesos]) - 1

    #     ddf = dd.from_pandas(df, npartitions=50)
    #     with ProgressBar():
    #         df = ddf.groupby(['PERSONA_REF_ID']).mean().compute()

        # Editar columnas
        df['P10'] = 2 - df['P10']
        df['P09'] = df.P09.replace(5, 4) #Polimodal tomado como secundario 

    #     df = df.astype(int)
        df['P0910'] = df.P09.astype(str) + df.P10.astype(str)
        df['Grupo_Etario_3'] = pd.cut(df.P03, np.arange(-1, 80, 3))#.round(-1)
        df['Grupo_Etario_INDEC'] = pd.cut(df.P03, np.array([0, 13, 29, 64, 110]))#.round(-1)
        df['Grupo_Etario_q10'] = pd.cut(df.P03, np.array([-0.001, 5.0,  11.0, 17.0, 23.0, 29.0, 36.0, 44.0, 53.0, 65.0, 110.0]))#.round(-1)


        df = df.reset_index()

        df_cb = df.merge(ad_eq).merge(DPTO_Region).merge(CB_ipc)

        df_cb_hogares = df_cb.groupby(['HOGAR_REF_ID', 'Q'])[['P47T','CBA', 'CBT', 'CB_EQUIV']].sum()
        df_cb_hogares['Pobreza'] = df_cb_hogares['P47T'] < df_cb_hogares['CBT']
        df_cb_hogares['Indigencia'] = df_cb_hogares['P47T'] < df_cb_hogares['CBA']
        pobreza_hogares = df_cb_hogares[['P47T','CBA','CBT', 'CB_EQUIV','Pobreza', 'Indigencia']].reset_index()
        pobreza_hogares['gap_pobreza'] = pobreza_hogares.P47T - pobreza_hogares.CBT
        pobreza_hogares['gap_indigencia'] = pobreza_hogares.P47T - pobreza_hogares.CBA
        pobreza_hogares = pobreza_hogares.rename(columns = {'P47T': 'P47T_hogar'})

        # df = df.sample(25000)
        data = df.merge(pobreza_hogares, on = ['HOGAR_REF_ID', 'Q'])#, how = 'left')

        data = data.rename(columns = {'P47T': 'P47T_persona'})

        # data = data
        data = data.merge(radio_ref[['RADIO_REF_ID', 'IDFRAC', 'PROV', 'NOMPROV', 'AGLOMERADO', 'Region']].drop_duplicates())

        print("Poblacion: "+str(len(data)/frac))

        data.to_csv(filename, index = False) ## Aca si ya existen no deberian sobreescribirse (o si)

2019-02-15
Poblacion: 44210000.0
2019-05-15
Poblacion: 44210000.0
2019-08-15
Poblacion: 44210000.0
2019-11-15
Poblacion: 44210000.0
2020-02-15
Poblacion: 44603650.0
2020-05-15
Poblacion: 44603650.0
2020-08-15
Poblacion: 44603650.0


In [21]:
data.columns

Index(['index', 'RADIO_REF_ID', 'DPTO', 'HOGAR_REF_ID', 'H15', 'H16', 'IX_TOT',
       'PERSONA_REF_ID', 'P02', 'P03', 'P09', 'P10', 'CONDACT', 'CAT_OCUP',
       'CAT_INAC', 'P47T_persona', 'ANO4', 'Q', 'P0910', 'Grupo_Etario_3',
       'Grupo_Etario_INDEC', 'Grupo_Etario_q10', 'P47T_hogar', 'CBA', 'CBT',
       'CB_EQUIV', 'Pobreza', 'Indigencia', 'gap_pobreza', 'gap_indigencia',
       'IDFRAC', 'PROV', 'NOMPROV', 'AGLOMERADO', 'Region'],
      dtype='object')

# Distribucion del ingreso

In [18]:
# import numpy as np
# fig, ax = plt.subplots(1, figsize = (5, 3))
# plt.hist(np.log10(df.loc[df.P47T > 100].P47T).values, 150, normed = 'pdf')#.groupby()
# plt.yscale('log')
# plt.show()

In [19]:
# # fig, ax = plt.subplots(1, figsize = (10, 5))
# fig, ax = plt.subplots(1, figsize = (5, 3))
# df_ = df.groupby('HOGAR_REF_ID').sum()#.loc[df.P47T > 100]
# agg = df_.groupby(pd.cut(np.log10(df_.P47T), np.arange(2, 6, .05)))['P47T'].agg(['count','sum'])
# (agg/agg.sum()).plot(ax = ax)
# # ax.set_yscale('log')